In [1]:
import sys
sys.path.append("..")
from VAE.vae import *

In [2]:
config_space = ConfigurationSpace(
                {'input_dropout': 0.1, 'intermediate_activation': "relu", 'intermediate_dimension': 10,
                'intermediate_layers': 2, 'latent_dimension': 1, 'learning_rate': 0.0001,
                'original_dim': 825000, 'solver': 'nadam'}
            )
config = config_space.get_default_configuration()

In [3]:
model = FIA_VAE(config)

In [4]:
model.summary()

Model: "fia_vae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_intermediate            │ ?                      │     8,250,065 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_mu (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_sigma (Dense)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_reparametrization        │ ?                      │   0 (unbuilt) │
│ (Sampling)                      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ ?                      │     9,075,070 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,325,135 (66.09 MB)

 Trainable params: 17,325,135 (66.09 MB)

 Non-trainable params: 0 (0.00 B)

## Saving

In [5]:
keras.saving.save_model(model, "../../runs/VAE/training/test.keras")

/mnt/lustre/groups/link/linca945/.conda/envs/FIA_VAE/lib/python3.11/site-packages/keras/src/saving/saving_api.py:100: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  saving_lib.save_model(model, filepath)


In [6]:
model.save_weights("../../runs/VAE/training/test.weights.h5")

## Loading

In [7]:
model2 = keras.saving.load_model("../../runs/VAE/training/test.keras")

In [8]:
model2.load_weights("../../runs/VAE/training/test.weights.h5")

## Testing

In [9]:
data_dir  = os.path.normpath(os.path.join(os.getcwd(), "../../runs/FIA/Com8_grown_together/oms"))
data = read_data(data_dir, verbosity=0)

In [10]:
X = np.random.normal(0.0, 1.0, size=(5, 825000))

In [13]:
callbacks = [ keras.callbacks.ModelCheckpoint( filepath="../../runs/VAE/training/test_{epoch}.keras",
                                                       save_best_only=True, monitor="val_loss",
                                                       verbose=1 )]
callbacks.append( )

TypeError: list.append() takes exactly one argument (0 given)

In [14]:
model2.fit(X, X, validation_split=0.2, batch_size=2, epochs=10, callbacks=callbacks)

Epoch 1/10


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - kl_loss: 0.7328 - loss: 1.5298 - reconstruction_loss: 0.7970
Epoch 1: val_loss improved from inf to 0.84213, saving model to ../../runs/VAE/training/test_1.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step - kl_loss: 0.7328 - loss: 1.5298 - reconstruction_loss: 0.7970 - val_kl_loss: 0.0447 - val_loss: 0.8421 - val_reconstruction_loss: 0.7974
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - kl_loss: 13287.1094 - loss: 13287.9062 - reconstruction_loss: 0.7974
Epoch 2: val_loss improved from 0.84213 to 0.82966, saving model to ../../runs/VAE/training/test_2.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step - kl_loss: 13287.1094 - loss: 13287.9062 - reconstruction_loss: 0.7974 - val_kl_loss: 0.0322 - val_loss: 0.8297 - val_reconstruction_loss: 0.7974
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - kl_loss: 804.3394 - loss: 805.1360 - reconstruction_loss: 0.7965
Epoch 3: val_loss did not improve from 0.82966
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - kl_l

In [15]:
loss, kl_loss, recon_loss = model2.evaluate(data, data, batch_size=68)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - kl_loss: 2.8402e-05 - loss: 7.4781e-04 - reconstruction_loss: 7.1941e-04


In [17]:
prediction = model2.predict(data)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [19]:
pd.DataFrame(prediction)

,0,1,2,3,4,5,6,7,8,9,...,824990,824991,824992,824993,824994,824995,824996,824997,824998,824999
0,0.000000,0.000271,0.003579,0.000000,0.0,0.000393,0.000000,0.000000,0.002761,0.002984,...,0.003010,0.0,0.000412,0.000000,0.001058,0.000000,0.000177,0.0,0.000000,0.001853
1,0.000783,0.000236,0.002461,0.004321,0.0,0.000563,0.000610,0.000000,0.001099,0.001626,...,0.002565,0.0,0.000435,0.002737,0.001041,0.001859,0.001998,0.0,0.000127,0.000000
2,0.000000,0.000063,0.002080,0.001112,0.0,0.000636,0.000000,0.000000,0.000452,0.001553,...,0.001434,0.0,0.000621,0.000415,0.001264,0.000276,0.000636,0.0,0.000105,0.000729
3,0.000000,0.000367,0.004271,0.000000,0.0,0.000281,0.000000,0.000000,0.003827,0.003644,...,0.003738,0.0,0.000316,0.000000,0.000962,0.000000,0.000000,0.0,0.000000,0.002372
4,0.001032,0.000315,0.002708,0.005441,0.0,0.000519,0.000908,0.000120,0.001505,0.001754,...,0.003100,0.0,0.000350,0.003487,0.000943,0.002281,0.002476,0.0,0.000112,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.000791,0.000239,0.002469,0.004357,0.0,0.000562,0.000619,0.000000,0.001113,0.001631,...,0.002583,0.0,0.000432,0.002762,0.001038,0.001873,0.002014,0.0,0.000127,0.000000
64,0.000000,0.000351,0.004160,0.000000,0.0,0.000299,0.000000,0.000000,0.003655,0.003538,...,0.003621,0.0,0.000332,0.000000,0.000977,0.000000,0.000000,0.0,0.000000,0.002288
65,0.000000,0.000068,0.002114,0.001086,0.0,0.000630,0.000000,0.000000,0.000505,0.001585,...,0.001470,0.0,0.000616,0.000376,0.001259,0.000221,0.000625,0.0,0.000097,0.000754
66,0.000000,0.000631,0.006178,0.000000,0.0,0.000000,0.000000,0.000000,0.006763,0.005464,...,0.005743,0.0,0.000051,0.000000,0.000699,0.000000,0.000000,0.0,0.000000,0.003802
